# IMPORTING DATA

In [4]:
# Importing libraries
import pandas as pd
from sqlalchemy import create_engine

## Acquiring data from db. file to DataFrame:

In [5]:
# Using sqlalchemy to connect with the file which contains the  messy dataset Forbes Billionaires 2018
engine = create_engine("sqlite:///../data/raw/CristopherRL.db")
#engine = create_engine('http://potacho.com/files/ironhack/CristopherRL.db')

In [6]:
# Creating a query to incorporate all 3 tables in one with all the necessary information
query = """ 
SELECT 
personal_info.id, 
position, 
lastName, 
rank_info.name, 
age, 
personal_info."Unnamed: 0", 
gender, 
country, 
image, 
business_info.Source,
business_info.worth,
business_info.worthChange,
business_info.realTimeWorth
FROM personal_info
LEFT JOIN rank_info     ON personal_info.id = rank_info.id
LEFT JOIN business_info ON personal_info.id = business_info.id
ORDER BY position
;
"""

In [7]:
# Importing data from db file to dataframe
raw_data = pd.read_sql_query(query, engine)
raw_data.head(5)

,id,position,lastName,name,age,Unnamed: 0,gender,country,image,Source,worth,worthChange,realTimeWorth
0,8254,1.0,bEZOS,jefF BEZOS,54 years old,52,Male,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None
1,6688,2.0,gaTEs,bill GATES,62 years old,53,None,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None
2,1904,3.0,bUFFETT,WARREN BUffett,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,None
3,4168,4.0,aRNAULT,bernARD Arnault,69 years old,55,M,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,None
4,7545,5.0,zuckeRBERG,maRK ZUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,None


# WRANGLING DATA

In [8]:
import re

In [9]:
#proc_data DataFrame will be the clean dataset
proc_data = raw_data

In [10]:
#rows and columns
proc_data.shape

(2208, 13)

## Analizing Series:

In [11]:
proc_data.columns

Index(['id', 'position', 'lastName', 'name', 'age', 'Unnamed: 0', 'gender',
       'country', 'image', 'Source', 'worth', 'worthChange', 'realTimeWorth'],
      dtype='object')

In [12]:
#Analyzing the type of every column
proc_data.dtypes
#There are many columns to change the type, but firstly it's neccesary to change them values

id                 int64
position         float64
lastName          object
name              object
age               object
Unnamed: 0         int64
gender            object
country           object
image             object
Source            object
worth             object
worthChange       object
realTimeWorth     object
dtype: object

In [13]:
null_cols = proc_data.isnull().sum()
null_cols[null_cols > 0]
#all values in realTimeWorth are nulls > DELETE THIS COLUMN

age                65
gender             13
realTimeWorth    2208
dtype: int64

#### > Ages

In [14]:
null_age = proc_data[(proc_data['age'].isnull()==True)]
null_age #There is no problem to delete 65 rows

,id,position,lastName,name,age,Unnamed: 0,gender,country,image,Source,worth,worthChange,realTimeWorth
54,1393,55.0,HINDUJA,hiNDUJA Family,None,106,None,None,https://specials-images.forbesimg.com/imageser...,Diversified ==> diversified,19.5 BUSD,0.001 millions USD,None
76,2609,77.0,KWOK,thomaS & RAYMOND kwok,None,128,Male,Hong Kong,https://specials-images.forbesimg.com/imageser...,Real Estate ==> real estate,16.5 BUSD,0.0 millions USD,None
89,6060,90.0,reUBEN,dAVID & SIMON REUBen,None,141,M,None,https://specials-images.forbesimg.com/imageser...,"Real Estate ==> investments, real estate",15.5 BUSD,0.0 millions USD,None
129,7458,130.0,TSCHIRA,udo & HARALD TSCHIRA,None,181,Male,None,https://specials-images.forbesimg.com/imageser...,Technology ==> software,11.7 BUSD,0.0 millions USD,None
141,3572,142.0,NG,ROBERT & PHIlip ng,None,193,M,None,https://specials-images.forbesimg.com/imageser...,Real Estate ==> real estate,10.8 BUSD,0.0 millions USD,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,1160,2021.0,chIARAVANOND,yupa chiARavanond,None,2072,F,None,https://specials-images.forbesimg.com/imageser...,Diversified ==> diversified,1.1 BUSD,0.0 millions USD,None
2021,2877,2022.0,CHIRAVAnond,PRATHIP CHIRAVanond,None,2073,Male,None,https://specials-images.forbesimg.com/imageser...,Diversified ==> diversified,1.1 BUSD,0.0 millions USD,None
2028,3859,2029.0,EGGEr,wolFGANG EGGER,None,2080,M,None,https://specials-images.forbesimg.com/imageser...,Real Estate ==> real estate,1.1 BUSD,nan millions USD,None
2044,6241,2045.0,kaPUr,madhu KApur,None,2096,None,None,https://specials-images.forbesimg.com/imageser...,Finance and Investments ==> banking,1.1 BUSD,0.0 millions USD,None


In [15]:
snone_age = proc_data[(proc_data['age']=='None')]
snone_age.shape #No ages with 'None' as string
## Finally, 65 rows of Ages are None

(0, 13)

#### > Gender

In [16]:
null_gender = proc_data[(proc_data['gender'].isnull()==True)]
null_gender.sort_values(by=['position']) #

,id,position,lastName,name,age,Unnamed: 0,gender,country,image,Source,worth,worthChange,realTimeWorth
26,4840,27.0,albreCHT JR.,beate heistER & KARL ALBRECHt jr.,66 years old,78,None,Germany,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> supermarkets,29.8 BUSD,0.0 millions USD,None
163,4493,164.0,APONTE,gianluigI & RAFAELA aponte,77 years old,215,None,Switzerland,https://specials-images.forbesimg.com/imageser...,Logistics ==> Shipping,9.3 BUSD,0.0 millions USD,None
224,5933,225.0,lIEBHErr,wiLLI & ISOLDE liebherr,None,276,None,None,https://specials-images.forbesimg.com/imageser...,Construction & Engineering ==> Construction,7.1 BUSD,0.0 millions USD,None
309,2698,310.0,lOVE,tom & JUDy love,81 years old,361,None,United States,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> retail & gas stations,5.8 BUSD,0.0 millions USD,None
426,8092,427.0,GREEN,philip & cRISTINa green,66 years old,478,None,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> fashion retail,4.8 BUSD,0.0 millions USD,None
529,6897,530.0,DANGERMOND,jaCK & LAURA Dangermond,72 years old,581,None,None,https://specials-images.forbesimg.com/imageser...,Technology ==> mapping software,4.1 BUSD,0.0 millions USD,None
581,3503,582.0,resNICK,stewart and LYNDA Resnick,None,633,None,None,https://specials-images.forbesimg.com/imageser...,"Food and Beverage ==> agriculture, water",3.9 BUSD,0.0 millions USD,None
596,5060,597.0,kuLCZYk,dominika & seBASTIAN KULCZYK,None,648,None,None,https://specials-images.forbesimg.com/imageser...,Finance and Investments ==> diversified,3.8 BUSD,0.001 millions USD,None
731,7708,732.0,cheRNG,andrEW & PEGGY CHERNG,None,783,None,None,https://specials-images.forbesimg.com/imageser...,Food and Beverage ==> restaurants,3.2 BUSD,0.0 millions USD,None
766,1566,767.0,chANg,do won & JIN SOOK Chang,62 years old,818,None,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> fashion retail,3.1 BUSD,0.0 millions USD,None


In [17]:
null_gender.shape #13 values in gender with 'None' values, but .....

(13, 13)

In [18]:
snone_gender = proc_data[(proc_data['gender']=='None')]
snone_gender

,id,position,lastName,name,age,Unnamed: 0,gender,country,image,Source,worth,worthChange,realTimeWorth
1,6688,2.0,gaTEs,bill GATES,62 years old,53,None,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None
9,1718,10.0,ELLISON,laRRY ELLISon,73 years old,61,None,United States,https://specials-images.forbesimg.com/imageser...,Technology ==> software,58.5 BUSD,-0.001 millions USD,None
12,6204,13.0,bRIN,seRGEY brin,44 years old,64,None,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Google,47.5 BUSD,0.0 millions USD,None
17,5166,18.0,bettencouRT MEYERS,FRANCOISE BETTENcourt meyers,64 years old,69,None,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> L'Oreal,42.2 BUSD,0.0 millions USD,None
18,6778,19.0,aMBANI,MUKESH AMBANi,61 years old,70,None,None,https://specials-images.forbesimg.com/imageser...,"Energy ==> petrochemicals, oil & gas",40.1 BUSD,0.0 millions USD,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,5515,2188.0,stAWSki,axel STAWSKI,67 years old,2239,None,None,https://specials-images.forbesimg.com/imageser...,Real Estate ==> real estate,1.0 BUSD,0.0 millions USD,None
2190,2441,2191.0,SUN,sun shaNGCHUAN,54 years old,2242,None,None,https://specials-images.forbesimg.com/imageser...,Technology ==> electronics components,1.0 BUSD,nan millions USD,None
2193,5519,2194.0,uPATKOOn,suRIN UPATKOON,69 years old,2245,None,None,https://specials-images.forbesimg.com/imageser...,"Diversified ==> telecom, lotteries, insurance",1.0 BUSD,0.0 millions USD,None
2201,5863,2202.0,yaNG,yaNG JIANXin,49 years old,2253,None,None,https://specials-images.forbesimg.com/imageser...,Technology ==> apparel,1.0 BUSD,nan millions USD,None


In [19]:
snone_gender.shape # ... there are 552 rows with 'None' as a string 

(552, 13)

## Changing data:

#### > Age

In [20]:
###### AGE
def real_age(x):
    
    if x is None:
        return 999 #to identify which doesnt have age informatio
    
    else:
        y = re.findall('[\d]+',x) # extracting y from "y years". The last on is '99 years'
        if int(y[0])<100:
            return int(y[0])
        else:
            return 2019-int(y[0]) #there are some people with year of birth
        
#applying function in every value and changing to int format
proc_data['n_age'] = proc_data['age'].apply(real_age).astype('int64')

#### > Gender

In [21]:
#when there is no information, it is filled with 'Unknown'
proc_data['gender'] = proc_data['gender'].fillna("Unknown")
#There are some None as string
proc_data.loc[proc_data['gender']=='None', 'gender'] = "Unknown"
#Replacin M for Male, and F for Female
proc_data.loc[proc_data['gender']=='M', 'gender'] = "Male"
proc_data.loc[proc_data['gender']=='F', 'gender'] = "Female"


In [22]:
#being sure that there are just 3 values
list(proc_data['gender'].unique())

['Male', 'Unknown', 'Female']

#### > Name

In [23]:
#To avoid differences in names, every name in upper letters
proc_data['FullName'] = proc_data['name'].str.upper()

###### LAST NAME > There is no reason to keep the Last Name if I have the Full Name
#proc_data['LastName'] = proc_data['lastName'].str.upper()


#### > Country

In [24]:
###### COUNTRY
proc_data['country'].value_counts()
#Checking values in countries > 1415/2208 = 64% null values > I'll look for this information on internet

None                          1415
United States                  177
China                          113
USA                             57
India                           45
People's Republic of China      44
Russia                          35
Germany                         34
Hong Kong                       26
Brazil                          19
South Korea                     15
Italy                           15
France                          14
United Kingdom                  12
Switzerland                     11
Turkey                          11
Thailand                        11
Canada                          11
Sweden                          11
Singapore                       10
Australia                       10
Israel                          10
Taiwan                           8
Indonesia                        7
Japan                            7
Philippines                      6
Spain                            6
UK                               6
Mexico              

In [25]:
null_country = proc_data[(proc_data['country']=='None')] #Null are as string
null_country

,id,position,lastName,name,age,Unnamed: 0,gender,country,image,Source,worth,worthChange,realTimeWorth,n_age,FullName
0,8254,1.0,bEZOS,jefF BEZOS,54 years old,52,Male,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None,54,JEFF BEZOS
1,6688,2.0,gaTEs,bill GATES,62 years old,53,Unknown,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None,62,BILL GATES
3,4168,4.0,aRNAULT,bernARD Arnault,69 years old,55,Male,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,None,69,BERNARD ARNAULT
4,7545,5.0,zuckeRBERG,maRK ZUCKERBERG,1985,56,Male,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,None,34,MARK ZUCKERBERG
5,2790,6.0,ortEGA,AMANCIO ORTEGa,82 years old,57,Male,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> Zara,70.0 BUSD,0.0 millions USD,None,82,AMANCIO ORTEGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,5247,2201.0,WU,WU CHUNG-YI,59 years old,2252,Male,None,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> manufacturing,1.0 BUSD,0.0 millions USD,None,59,WU CHUNG-YI
2201,5863,2202.0,yaNG,yaNG JIANXin,49 years old,2253,Unknown,None,https://specials-images.forbesimg.com/imageser...,Technology ==> apparel,1.0 BUSD,nan millions USD,None,49,YANG JIANXIN
2203,2301,2204.0,zHAO,zhao xiAOQIANG,51 years old,2255,Unknown,None,https://specials-images.forbesimg.com/imageser...,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,None,51,ZHAO XIAOQIANG
2204,5611,2205.0,zhOU,ZHOU LIANGZHAng,55 years old,2256,Male,None,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,None,55,ZHOU LIANGZHANG


##### > WORTH

In [ ]:
proc_data['worth_BUSD'] = proc_data['worth'].str.replace(' BUSD','')

##### > WORTHCHANGE

In [26]:
proc_data['worthChange_MUSD'] = proc_data['worthChange'].str.replace(' millions USD','')

#### > UNNAMED: 0

In [27]:

#Understanding the different with position column
proc_data['dif'] = proc_data['Unnamed: 0'] -  proc_data['position']
list(proc_data['dif'].unique())
#These columns have the same difference

[51.0]

#### > Source

In [28]:

#proc_data['Industry'] = proc_data['source'].str.split(' ==>')[0]

def ind(x):
    y = x.split(' ==> ')
    return y[0] #taking first element of the list

def comp(x):
    y = x.split(' ==> ')
    return y[1] #taking second element of the list

#applying function in every value and changing to int format
proc_data['Industry'] = proc_data['Source'].apply(ind)
proc_data['Company']  = proc_data['Source'].apply(comp)

### Deleting columns:

In [29]:
proc_data.drop(columns=['dif','lastName','name','country','Source','age','Unnamed: 0','worth','worthChange','realTimeWorth'], inplace=True)

### Changing data types:

In [30]:
##################  DATA TYPES ##################
#Changing every column to the correct type
proc_data['id']       = proc_data['id'].astype('object') 
proc_data['position'] = proc_data['position'].astype('int64')
proc_data['worth_BUSD'] = proc_data['worth_BUSD'].astype('float64')
proc_data['worthChange_MUSD'] = proc_data['worthChange_MUSD'].astype('float64')

In [31]:
proc_data.dtypes

id                   object
position              int64
gender               object
image                object
n_age                 int64
FullName             object
worth_BUSD          float64
worthChange_MUSD    float64
Industry             object
Company              object
dtype: object

In [32]:
proc_data.describe()

,position,n_age,worth_BUSD,worthChange_MUSD
count,2208.000000,2208.000000,2208.000000,2101.000000
mean,1104.500000,91.342844,4.103080,-0.484737
std,637.539018,158.638692,6.913084,26.330360
min,1.000000,21.000000,1.000000,-672.311000
25%,552.750000,54.000000,1.400000,0.000000
50%,1104.500000,64.000000,2.200000,0.000000
75%,1656.250000,74.000000,4.000000,0.000000
max,2208.000000,999.000000,112.000000,530.206000


# Processed data file

In [33]:
new_order = ['position','id','FullName','worth_BUSD','worthChange_MUSD','Industry','Company','n_age','gender','image']

In [34]:
proc_data = proc_data[new_order]

In [35]:
proc_data.head(10)

,position,id,FullName,worth_BUSD,worthChange_MUSD,Industry,Company,n_age,gender,image
0,1,8254,JEFF BEZOS,112.0,0.000,Technology,Amazon,54,Male,https://specials-images.forbesimg.com/imageser...
1,2,6688,BILL GATES,90.0,-0.001,Technology,Microsoft,62,Unknown,https://specials-images.forbesimg.com/imageser...
2,3,1904,WARREN BUFFETT,84.0,-0.002,Finance and Investments,Berkshire Hathaway,87,Male,https://specials-images.forbesimg.com/imageser...
3,4,4168,BERNARD ARNAULT,72.0,0.000,Fashion & Retail,LVMH,69,Male,https://specials-images.forbesimg.com/imageser...
4,5,7545,MARK ZUCKERBERG,71.0,0.000,Technology,Facebook,34,Male,https://specials-images.forbesimg.com/imageser...
5,6,2790,AMANCIO ORTEGA,70.0,0.000,Fashion & Retail,Zara,82,Male,https://specials-images.forbesimg.com/imageser...
6,7,8711,CARLOS SLIM HELU,67.1,0.001,Telecom,telecom,78,Male,https://specials-images.forbesimg.com/imageser...
7,8,1066,CHARLES KOCH,60.0,0.000,Diversified,Koch Industries,82,Male,https://specials-images.forbesimg.com/imageser...
8,9,5843,DAVID KOCH,60.0,0.000,Diversified,Koch Industries,78,Male,https://specials-images.forbesimg.com/imageser...
9,10,1718,LARRY ELLISON,58.5,-0.001,Technology,software,73,Unknown,https://specials-images.forbesimg.com/imageser...


In [36]:
#Exporting clean data to csv
proc_data.to_csv('../data/processed/CristopherRL_processed_data.csv', sep='|', index=False)

In [37]:
############# END  ############